In [1]:
#https://www.tensorflow.org/tutorials/eager/custom_layers
import tensorflow as tf
tfe = tf.contrib.eager

tf.enable_eager_execution()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(5, input_shape=(None, 3))

In [3]:
# To use a layer, simply call it.
layer(tf.zeros([2, 3]))

<tf.Tensor: id=30, shape=(2, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer by calling layer.variables. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(3, 5) dtype=float32, numpy=
 array([[-0.03726137,  0.49221903, -0.03322661,  0.30009776,  0.5343706 ],
        [ 0.14106745,  0.6720144 ,  0.13084334, -0.14323545,  0.6951353 ],
        [ 0.5216983 ,  0.55106586, -0.6370045 , -0.8315506 ,  0.57707304]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>]

In [5]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(3, 5) dtype=float32, numpy=
 array([[-0.03726137,  0.49221903, -0.03322661,  0.30009776,  0.5343706 ],
        [ 0.14106745,  0.6720144 ,  0.13084334, -0.14323545,  0.6951353 ],
        [ 0.5216983 ,  0.55106586, -0.6370045 , -0.8315506 ,  0.57707304]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>)

In [22]:
"""Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:
__init__ , where you can do all input-independent initialization
build, where you know the shapes of the input tensors and can do the rest of the initialization
call, where you do the forward computation
Note that you don't have to wait until build is called to create your variables, you can also create them in __init__. 
However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. 
On the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.
"""

"Implementing custom layers\nThe best way to implement your own layer is extending the tf.keras.Layer class and implementing:\n__init__ , where you can do all input-independent initialization\nbuild, where you know the shapes of the input tensors and can do the rest of the initialization\ncall, where you do the forward computation\nNote that you don't have to wait until build is called to create your variables, you can also create them in __init__. \nHowever, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. \nOn the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.\n"

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[input_shape[-1].value, 
                                           self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(4)
print(layer(tf.zeros([10, 5])))
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]], shape=(10, 4), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 4) dtype=float32, numpy=
array([[ 0.3275516 ,  0.6117302 , -0.10604715,  0.7484832 ],
       [ 0.7489594 ,  0.5077696 ,  0.02631444, -0.06587273],
       [ 0.03284341,  0.6599947 , -0.3738828 ,  0.34896195],
       [-0.04443628, -0.56095076,  0.23255992, -0.26508248],
       [ 0.75677407, -0.5686913 , -0.4760599 ,  0.20946705]],
      dtype=float32)>]


In [23]:
"""Models: composing layers
Many interesting layer-like things in machine learning models are implemented by composing existing layers. 
For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut.
The main class used when creating a layer-like thing which contains other layers is tf.keras.Model. 
Implementing one is done by inheriting from tf.keras.Model.
"""

'Models: composing layers\nMany interesting layer-like things in machine learning models are implemented by composing existing layers. \nFor example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut.\nThe main class used when creating a layer-like thing which contains other layers is tf.keras.Model. \nImplementing one is done by inheriting from tf.keras.Model.\n'

In [24]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0', 'resnet_identity_block/batch_normalization/moving_mean:0', 'resnet_identity_block/batch_normalization/moving_variance:0', 'resnet_identity_block/batch_normalization_1/moving_mean:0', 'resnet_identity_block/batch_normalization_1/moving_variance:0', 'resnet_identity_block/batch_normalization_2